# Preprocessing
Reading the data, cleaning and storing into database


## Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sqlite3 as sql
from IPython.display import display

## Data Functions 

In [2]:
def get_session_details_df(clm,courseCode):
    '''
    extrating details of the session from the columnn names
    
    input:
        clm:the columnn names, list type 
        courseCode: course code, string type
    return: 
        dfComponetns: the session details, dataframe type       
    '''
    
    dfComponents=pd.DataFrame(clm,columns=["details"])
    dfComponents=dfComponents["details"].str.split("\n",expand = True) 
    last_row=len(dfComponents.columns)
    dfComponents.drop(last_row-1,inplace=True, axis=1)
    dfComponents.columns=["session_no","day","time","session_type","location"]
    dfComponents.index.names=["a_id"]
    dfComponents['course'] = courseCode
    dfComponents['week_no'] = dfComponents["session_no"].apply(lambda x: pd.Series(str(x).split(".")[1][1]))
    dfComponents['week_no'] = dfComponents['week_no'].astype('int')
    
    return dfComponents

def get_sessionid(dfCOASessions):
    '''
    create a unique session ID for each session
    
    input:
        dfCOASessions:the session details, dataframe type 
    return: 
        dfCOASessions: the session details with session ID, dataframe type    
    '''
    
    x=[]
    for index, row in dfCOASessions.iterrows():
        day= row['day'][:3]
        start_time, end_time = row['time'].split("-")
        start_hour = start_time.split(":")[0]
        end_hour = end_time.split(":")[0]
        time = start_hour.strip()+end_hour.strip()
        if not row['location']:
            a=row['course']+'.'+row['session_no']+'.'+day\
                +'.'+time+'.'+row['session_type'].replace(" ", "")
        else:
            a=row['course']+'.'+row['session_no']+'.'+day\
                +'.'+time+'.'+row['session_type'].replace(" ", "")\
                +'.'+row['location'].replace("...", "")
        x.append(str(a))
    dfCOASessions.insert(0,'session_id', x)
    
    return dfCOASessions

def get_data(csv_filename):
    '''
     Read all Attendance from csv files
     
     input:
        csv_filename: csv file name, string type 
     Return two dataframes : 
         Student Attendance and the details of each session 
    '''
    
    file=csv_filename+".csv" # file name 
    course=csv_filename[2:8]
    path = "./data/" #Folder Path
    dfCOA=pd.read_csv(path+file,index_col=0)
    clm=dfCOA.columns.to_list()
    
    dfCOASessions=get_session_details_df(clm,course)
    dfCOASessions=get_sessionid(dfCOASessions)
    dfCOA.columns=dfCOASessions['session_id']
    dfCOA.index.names=["sid"]

 
    return (dfCOA,dfCOASessions)

In [3]:
def cleaning_data(dfCOA):
    '''
     Cleaning the data frame
     
     input:
        dfCOA: student attendance, dataframe type 
     Return two dataframes : 
         dfCleanCOA: Attendance, dataframe type 
    '''
    
    dfCOA.replace({'GPS': True, 'X': False, 'Ex':np.nan},inplace=True)
    selected_col =dfCOA.columns[dfCOA.isna().all()]    
    dfCOA.drop(selected_col, axis=1,inplace=True)
    selected_rows = dfCOA[dfCOA.isnull().all(axis=1)]
    dfCleanCOA=dfCOA.drop(selected_rows.index,axis=0)
    
    return (dfCleanCOA)

def combining_data(dfCOA111,dfCOA122):
    '''
    Making a single data frame form two courses
    '''
    
    dfCOA=pd.merge(dfCOA111,dfCOA122,how='inner',\
                   left_index=True,right_index=True)
    return dfCOA

### Database operations 

In [4]:
def write_to_db(dfData,TableName):
    '''
    write the dataframe to the DB 
    '''
    
    conn = sql.connect('CWDatabase.db')
    dfData.to_sql(TableName,if_exists='replace',
                  index=True,
                  con=conn) 
    conn.close()
    
def get_student_attendance_from_db():
    '''
    read db 
    returning student attendance
    '''
    
    conn = sql.connect('CWDatabase.db')
    query = "select * from dfCleanCOA"
    dfStuAtt=pd.read_sql(query,
                         con=conn,
                         index_col=["sid"])
    return dfStuAtt

def get_session_details_from_db():
    '''
    read db 
    returning the details of sessions 
    '''
    
    conn = sql.connect('CWDatabase.db')
    query = "select * from dfCOASessions"
    dfSession=pd.read_sql(query,
                          con=conn,
                          index_col=["a_id"])
    return dfSession

## Main Function

In [5]:
def main():
    '''
    main function
    '''
    
    # reading 
    dfCOA111,dfCOA111Sessions=get_data("22COA111ModuleRegister")
    dfCOA122,dfCOA122Sessions=get_data("22COA122ModuleRegister")
    
    #cleaning
    dfCOA = combining_data(dfCOA111,dfCOA122)
    dfCleanCOA = cleaning_data(dfCOA)
    dfCOASessions=dfCOA111Sessions.append(dfCOA122Sessions)

    # writing data into the DB
    write_to_db(dfCleanCOA,
                "dfCleanCOA")
    write_to_db(dfCOASessions,
                "dfCOASessions")

#calling main function  
main()

## Testing 

In [6]:
def get_tables():
    '''
    getting tables from database
    '''
    
    df_student_attendance=get_student_attendance_from_db()
    dfsessions_details = get_session_details_from_db()
    return (df_student_attendance,dfsessions_details)

#calling get_tables function
dfCleanCOA,dfCOASessions=get_tables()    

In [7]:
display(dfCleanCOA)

,COA111.S1.W1.Mon.0910.Lecture.U020,COA111.S1.W1.Wed.1011.Lecture.SMB014,COA111.S1.W2.Wed.1011.Lecture.SMB014,COA111.S1.W2.Thu.1415.Lecture.CC011,COA111.S1.W3.Wed.1011.Lecture.SMB014,COA111.S1.W3.Thu.1415.Lecture.CC011,COA111.S1.W4.Wed.1011.Lecture.SMB014,COA111.S1.W4.Thu.1415.Lecture.CC011,COA111.S1.W5.Wed.1011.Lecture.SMB014,COA111.S1.W5.Thu.1415.Lecture.CC011,...,COA122.S1.W3.Thu.0911.ComputerLab.N001,COA122.S1.W3.Thu.1113.ComputerLab.N001,COA122.S1.W4.Mon.1416.Lecture.CC012,COA122.S1.W4.Thu.0911.ComputerLab.N001,COA122.S1.W4.Thu.1113.ComputerLab.N001,COA122.S1.W4.Fri.1213.PersonalBest.CC011,COA122.S1.W5.Mon.1416.Lecture.CC012,COA122.S1.W5.Thu.0911.Test.N001,COA122.S1.W5.Thu.1113.Test.N001,COA122.S1.W6.Mon.1416.Lecture.CC012
sid,,,,,,,,,,,,,,,,,,,,,
0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,...,0.0,NaN,1.0,0.0,NaN,0.0,0.0,1.0,NaN,0.0
1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,NaN,1.0,1.0,NaN,1.0,1.0,1.0,NaN,1.0
2,1.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,1.0,NaN,1.0,0.0,NaN,0.0,0.0,1.0,NaN,1.0
3,NaN,NaN,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,...,NaN,1.0,0.0,NaN,1.0,1.0,1.0,NaN,1.0,1.0
4,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,...,0.0,NaN,NaN,NaN,NaN,1.0,1.0,0.0,NaN,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
238,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,...,1.0,NaN,1.0,0.0,NaN,1.0,1.0,1.0,NaN,1.0
239,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,...,0.0,NaN,0.0,1.0,NaN,1.0,1.0,1.0,NaN,0.0
240,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,...,NaN,1.0,1.0,NaN,1.0,1.0,1.0,NaN,1.0,1.0


In [8]:
display(dfCOASessions)

,session_id,session_no,day,time,session_type,location,course,week_no
a_id,,,,,,,,
0,COA111.S1.W1.Mon.0910.Lecture.U020,S1.W1,Monday (03-10-2022),09:00 - 10:00,Lecture,U020,COA111,1
1,COA111.S1.W1.Wed.1011.Lecture.SMB014,S1.W1,Wednesday (05-10-2022),10:00 - 11:00,Lecture,SMB014,COA111,1
2,COA111.S1.W2.Wed.1011.Lecture.SMB014,S1.W2,Wednesday (12-10-2022),10:00 - 11:00,Lecture,SMB014,COA111,2
3,COA111.S1.W2.Wed.1213.Tutorial,S1.W2,Wednesday (12-10-2022),12:00 - 13:00,Tutorial,,COA111,2
4,COA111.S1.W2.Thu.1415.Lecture.CC011,S1.W2,Thursday (13-10-2022),14:00 - 15:00,Lecture,CC011,COA111,2
5,COA111.S1.W3.Wed.1011.Lecture.SMB014,S1.W3,Wednesday (19-10-2022),10:00 - 11:00,Lecture,SMB014,COA111,3
6,COA111.S1.W3.Wed.1213.Tutorial,S1.W3,Wednesday (19-10-2022),12:00 - 13:00,Tutorial,,COA111,3
7,COA111.S1.W3.Thu.1415.Lecture.CC011,S1.W3,Thursday (20-10-2022),14:00 - 15:00,Lecture,CC011,COA111,3
8,COA111.S1.W4.Wed.1011.Lecture.SMB014,S1.W4,Wednesday (26-10-2022),10:00 - 11:00,Lecture,SMB014,COA111,4
